In [1]:
%pip install transformers
%pip install datasets
%pip install faiss
%pip install torch

Defaulting to user installation because normal site-packages is not writeable
DEPRECATION: distro-info 1.1build1 has a non-standard version number. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of distro-info or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063
DEPRECATION: python-debian 0.1.43ubuntu1 has a non-standard version number. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of python-debian or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
DEPRECATION: distro-info 1.1build1 has a non-standard version number. pi

In [2]:
import pandas as pd
import re
import nltk
from datasets import load_dataset

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer

from keras.preprocessing.text import Tokenizer
from transformers import AutoTokenizer, AutoModel

from transformers import BertTokenizer, BertModel
import torch

/home/vip/victor.henrique/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-11-27 11:35:31.861883: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-27 11:35:33.925499: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-27 11:35:33.927119: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-27 11:35:41.532107: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [3]:
dataset = load_dataset("talanAI/resumesamples")
dataset = pd.DataFrame(dataset['train'])
dataset = dataset.dropna()
dataset = dataset[:100]
print(dataset.shape)

/home/vip/victor.henrique/.local/lib/python3.10/site-packages/huggingface_hub/repocard.py:105: UserWarning: Repo card metadata block was not found. Setting CardData to empty.
  warnings.warn("Repo card metadata block was not found. Setting CardData to empty.")


(100, 3)


In [4]:
# Load the BERT model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

def get_document_vector(doc):
    # Tokenize the document
    tokens = tokenizer(doc, return_tensors='pt', truncation=True, padding=True)

    # Get the embeddings from the last layer of BERT
    outputs = model(**tokens)
    embeddings = outputs.last_hidden_state

    # Calculate the mean of the embeddings
    doc_vector = torch.mean(embeddings, dim=1)
    return doc_vector.squeeze()

# Get vectors for documents
doc1 = "This is the first document."
doc2 = "This is the second document."
vector1 = get_document_vector(doc1)
vector2 = get_document_vector(doc2)

# dataset['Resume'] = dataset['Resume'].apply(preprocess_text)
# print(dataset['Resume'])